In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: ml_workspace_p1
Azure region: eastus
Subscription id: 76be00d8-3bef-43a6-89ba-2a12016e7f8c
Resource group: resourcegroup_p


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "Compute-Standard" #"compute-cluster"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS3_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

# get status of the cluster
print(cpu_cluster.get_status().serialize())

Found existing cluster, use it.

Running
{'errors': [], 'creationTime': '2022-05-31T21:00:38.655935+00:00', 'createdBy': {'userObjectId': 'f84c0cef-c750-41f6-8f8c-8ff6807766a4', 'userTenantId': 'd1ce8b91-bab1-49fc-937d-a779f008ae95', 'userName': None}, 'modifiedTime': '2022-05-31T21:03:27.999510+00:00', 'state': 'Running', 'vmSize': 'STANDARD_DS12_V2'}


In [3]:
# !pip3 install azureml.widgets
# pip install azureml-sdk[notebooks]
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler

parameter_space = {"C": choice(0.1,1,10), "max_iter": choice(50,100,150)}
ps = RandomParameterSampling(parameter_space = parameter_space)

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory = ".",
                            script='train.py',
                            compute_target=cluster_name,
                            environment = sklearn_env)


# est = SKLearn(source_directory=".", 
#               compute_target=cluster_name, 
#               vm_priority='lowpriority', 
#               entry_script='train.py')


# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
## ScriptRunConfig() should be passed to the run_config parameter instead of the estimator
hyperdrive_config = HyperDriveConfig(run_config=src,
                                     hyperparameter_sampling=ps,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal= PrimaryMetricGoal("MAXIMIZE"),
                                     max_total_runs=20,
                                     max_concurrent_runs=4,
                                     policy=policy)

In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(hyperdrive_config, show_output=True)
RunDetails(hyperdrive_run).show()

hyperdrive_run.wait_for_completion(show_output=True)

assert(hyperdrive_run.get_status() == "Completed")

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_3bbd2377-0822-4cdf-83e1-cdf7bcdbcc4f
Web View: https://ml.azure.com/runs/HD_3bbd2377-0822-4cdf-83e1-cdf7bcdbcc4f?wsid=/subscriptions/76be00d8-3bef-43a6-89ba-2a12016e7f8c/resourcegroups/resourcegroup_p/workspaces/ml_workspace_p1&tid=d1ce8b91-bab1-49fc-937d-a779f008ae95

Streaming azureml-logs/hyperdrive.txt

"<START>[2022-05-31T21:05:38.561065][API][INFO]Experiment created<END>\n""<START>[2022-05-31T21:05:39.650654][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n"<START>[2022-05-31T21:05:40.4162059Z][SCHEDULER][INFO]Scheduling job, id='HD_3bbd2377-0822-4cdf-83e1-cdf7bcdbcc4f_0'<END><START>[2022-05-31T21:05:40.5583241Z][SCHEDULER][INFO]Scheduling job, id='HD_3bbd2377-0822-4cdf-83e1-cdf7bcdbcc4f_1'<END>"<START>[2022-05-31T21:05:40.721517][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"<START>[2022-05-31T21:05:41.7278782Z][SCHEDULER][INFO]Successfully scheduled a job. Id='HD_3bbd2377-0822-4

In [5]:
import joblib
# Get your best run and save the model from that run.
# best_run = hyperdrive_run.get_best_run_by_primary_metric()
# best_run

best_model_metrics = hyperdrive_run.get_metrics()

print("Best run metrics :",best_model_metrics)

best_run_by_primary_metric = hyperdrive_run.get_best_run_by_primary_metric()
print("Best run by primary metric :",best_run_by_primary_metric)

#hyperdrive_run.get_children_sorted_by_primary_metric(top=1)

#joblib.dump(best_run, 'hd_bestrun_model.joblib')

Best run metrics : {'HD_3bbd2377-0822-4cdf-83e1-cdf7bcdbcc4f_8': {'Regularization Strength:': 1.0, 'Max iterations:': 100, 'Accuracy': 0.9109762266059687}, 'HD_3bbd2377-0822-4cdf-83e1-cdf7bcdbcc4f_7': {'Regularization Strength:': 0.1, 'Max iterations:': 100, 'Accuracy': 0.9105715730905413}, 'HD_3bbd2377-0822-4cdf-83e1-cdf7bcdbcc4f_5': {'Regularization Strength:': 10.0, 'Max iterations:': 100, 'Accuracy': 0.910065756196257}, 'HD_3bbd2377-0822-4cdf-83e1-cdf7bcdbcc4f_6': {'Regularization Strength:': 0.1, 'Max iterations:': 150, 'Accuracy': 0.9157309054122408}, 'HD_3bbd2377-0822-4cdf-83e1-cdf7bcdbcc4f_4': {'Regularization Strength:': 0.1, 'Max iterations:': 50, 'Accuracy': 0.9081436519979768}, 'HD_3bbd2377-0822-4cdf-83e1-cdf7bcdbcc4f_0': {'Regularization Strength:': 10.0, 'Max iterations:': 150, 'Accuracy': 0.906525037936267}, 'HD_3bbd2377-0822-4cdf-83e1-cdf7bcdbcc4f_2': {'Regularization Strength:': 1.0, 'Max iterations:': 150, 'Accuracy': 0.9123925139099646}, 'HD_3bbd2377-0822-4cdf-83e1-c

In [1]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(path=path)

In [2]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

import pandas as pd
data = pd.concat([x,y], axis=1)

In [3]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric="accuracy",
    training_data=data,
    label_column_name='y',
    n_cross_validations=5,
    )

In [4]:
#!pip show azureml-core
import azureml.core
print(azureml.core.VERSION)

import scipy
print(scipy.__version__)
#!pip install scipy==1.5.2


#!pip install azureml-train-automl==1.41.0

1.41.0
1.5.2


In [7]:
# Submit your automl run
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp=Experiment(ws, 'AutoML-project1')
remote_run = exp.submit(automl_config, show_output = True)
remote_run.wait_for_completion()

No run_configuration provided, running on local with default configuration
Running in the active local environment.


Experiment,Id,Type,Status,Details Page,Docs Page
AutoML-project1,AutoML_07d5dc9c-2618-42c6-a113-6e19c13437db,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one cl

2022-05-30:00:56:04,157 INFO     [explanation_client.py:334] Using default datastore for uploads


{'runId': 'AutoML_07d5dc9c-2618-42c6-a113-6e19c13437db',
 'target': 'local',
 'status': 'Completed',
 'startTimeUtc': '2022-05-30T00:22:33.537028Z',
 'endTimeUtc': '2022-05-30T00:54:09.169135Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'Experiment timeout reached, hence experiment stopped. Current experiment timeout: 0 hour(s) 30 minute(s)'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'local',
  'AMLSettingsJsonString': '{"path":null,"name":"AutoML-project1","subscription_id":"76be00d8-3bef-43a6-89ba-2a12016e7f8c","resource_group":"resourcegroup_p","workspace_name":"ml_workspace_p1","region":"eastus","compute_target":"local","spark_service":null,"azure_service":"Microsoft.AzureNotebookVM","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":fal

In [8]:
# Retrieve and save your best automl model.

best_run = remote_run.get_best_child()

best_run.get_details()

{'runId': 'AutoML_07d5dc9c-2618-42c6-a113-6e19c13437db_21',
 'target': 'local',
 'status': 'Completed',
 'startTimeUtc': '2022-05-30T00:52:38.452517Z',
 'endTimeUtc': '2022-05-30T00:53:15.67211Z',
 'services': {},
 'properties': {'runTemplate': 'automl_child',
  'pipeline_id': '__AutoML_Ensemble__',
  'pipeline_spec': '{"pipeline_id":"__AutoML_Ensemble__","objects":[{"module":"azureml.train.automl.ensemble","class_name":"Ensemble","spec_class":"sklearn","param_args":[],"param_kwargs":{"automl_settings":"{\'task_type\':\'classification\',\'primary_metric\':\'accuracy\',\'verbosity\':20,\'ensemble_iterations\':15,\'is_timeseries\':False,\'name\':\'AutoML-project1\',\'compute_target\':\'local\',\'subscription_id\':\'76be00d8-3bef-43a6-89ba-2a12016e7f8c\',\'region\':\'eastus\',\'spark_service\':None}","ensemble_run_id":"AutoML_07d5dc9c-2618-42c6-a113-6e19c13437db_21","experiment_name":null,"workspace_name":"ml_workspace_p1","subscription_id":"76be00d8-3bef-43a6-89ba-2a12016e7f8c","resource

### Cluster cleanup

In [6]:
cpu_cluster.delete()